In [11]:
import cuml
from cuml.svm import SVR
import numpy as np
from cuml.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score as CVS, train_test_split as TTS
import cupy as cp

import os
os.chdir("/local-scratch/GlucoseProject/mobicom23_mobispectral/regression")
import sys
sys.path.append(os.getcwd())
from utils import *
from evaluate import *

In [ ]:
data_root = '../datasets/dataset_skin/regression'
file_train = 'concatset_train.txt'
file_test = 'concatset_test.txt'
step = 1

In [ ]:
X_train, y_train, X_test, y_test, test_data = prepare_data(data_root, file_train, file_test, step)
# X_train, X_val, y_train, y_val = TTS(X_train, y_train, test_size=0.3, random_state=42)

In [ ]:
# Support Vector Regressor
svr_reg = SVR()
svr_reg.fit(X_train, y_train)
y_pred_svr = cp.asnumpy(svr_reg.predict(X_test))
print(f"ARD by signatures: {ARD(y_pred_svr, y_test).mean()}" )
evaluate_image(test_data, model=svr_reg)
ard = ard_image(test_data, svr_reg)
print("ARD by images: ", ard)
print("Average ARD: ", ard.mean())

In [ ]:
cv = KFold(n_splits=5, shuffle = True, random_state=42)

In [ ]:
# Parameter
model = SVR()

In [ ]:
axisx = range(180,220,1)
rs = []
for i in axisx:
    print(i)
    # model = XGBRegressor(n_estimators=i,random_state=42, tree_method='hist', device='cuda', n_jobs=-1)
    # rs.append(CVS(model,X_train,y_train,cv=cv,n_jobs=-1).mean())
    model.fit(X_train, y_train)
    # y_pred = model.predict(X_val)
    y_pred = cp.asnumpy(model.predict(X_test))
    val_r2_score = r2_score(y_test, y_pred)
    rs.append(val_r2_score.mean())

print(axisx[rs.index(max(rs))],max(rs))
# plt.figure(figsize=(20,5))
plt.plot(axisx,rs,c="red",label="XGB")
plt.legend()
plt.show()